### Парсинг hh и superjob. Заготовка из прошлого. Домашка далее...

In [70]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

user_input = 'Data science'

# user_input = input()
# if not user_input:
#     user_input = 'Data science'

# функция раздиения строки с зарплатой на МИН, МАКС и ВАЛЮТУ
def apd_salary(i):
    
    mi, ma = '', ''
    result = ['','','']
    
    i = i.replace(u'\xa0', u'')
    i = i.replace(u'.', u'')
    
    if re.search('от ', i):
        min = re.findall('\d+', i)
        mi = int(min[0])
        
    if re.search('до ', i):
        max = re.findall('\d+', i)
        ma = int(max[0])
 
    if re.search('\d+-\d+', i):
        min = re.findall('\d+', i)
        max = re.findall('-\d+', i)
        max[0] = max[0].replace(u'-', u'')
        mi = int(min[0])
        ma = int(max[0])
        
    result[0] = mi
    result[1] = ma
    result[2] = i[-3:]
    
    return(result) 


# САМА функция ПАРСИНГА
def parsing(html, site):
    res_block = html.find('div',{'class':'vacancy-serp'})
#     res_list = res_block.findChildren(recursive=False)

    results=[]
    for vak in res_block:
        data = {}
        vak_salary = ['','','']
        try:    
            vak_name = vak.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
            vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        except:
            continue           
    
        try:
            vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
            vak_salary = apd_salary(vak_s)
        except:
            pass

        data['name'] = vak_name
        data['salary_min'] = vak_salary[0]
        data['salary_max'] = vak_salary[1]
        data['salary_valuta'] = vak_salary[2]
        data['link'] = vak_link
        data['site'] = site

        results.append(data)    
        
    return(results)

main_link = 'https://hh.ru'
link = main_link + '/search/vacancy?clusters=true&enable_snippets=true&text=' + user_input
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

# СЧИТАЕМ СКРОЛЛИНГ
scr = html.findAll('a',{'class':'bloko-button HH-Pager-Control'})
scr = int(scr[-1].text)

i = 1
resultsHH=[]
while res_block:
    res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
    print(int(100*i/(scr+1)), '%', end=' ')
    resultsHH += parsing(html, main_link)
    link1 = link + '&page=' + str(i)
    i += 1
    response = requests.get(link1, headers=headers).text
    html = bs(response,'lxml')


print('hh - готово!')

# superjob.ru

main_link = 'https://www.superjob.ru'
link = main_link + '/vacancy/search/?keywords=' + user_input

response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

res_block = html.findAll('div',{'class':'f-test-vacancy-item'})
res_res = []

for vak in res_block:
    
    vak_link = vak.find('a',{'class':'icMQ_'})['href']
    vak_name = vak.find('div',{'class':'_3mfro'}).text
    vak_zar = vak.find('span',{'class':'f-test-text-company-item-salary'}).text
    
    mi, ma = '', ''
    data = {}
    vak_salary = ['','','']
    
    i = vak_zar.replace(u'\xa0', u'')
    
    if re.search('\d+', i):

        if re.search('^до', i):
            max = re.findall('\d+', i)
            ma = int(max[0])

        elif re.search('\d+—\d+', i):
            min = re.findall('\d+', i)
            max = re.findall('—\d+', i)
            max[0] = max[0].replace(u'—', u'')
            mi = int(min[0])
            ma = int(max[0])
        
        else:# re.search('от', i):
            min = re.findall('\d+', i)
            mi = int(min[0])
        
        vak_salary[0] = mi
        vak_salary[1] = ma
        vak_salary[2] = i[-1:]
    
    data['name'] = vak_name
    data['salary_min'] = vak_salary[0]
    data['salary_max'] = vak_salary[1]
    data['salary_valuta'] = vak_salary[2]
    data['link'] = vak_link
    data['site'] = main_link

    res_res.append(data)

print('superjob - сделано!')

print('hh: ', len(resultsHH))
print('superjob: ', len(res_res))

2 % 4 % 6 % 8 % 10 % 12 % 14 % 17 % 19 % 21 % 23 % 25 % 27 % 29 % 31 % 34 % 36 % 38 % 40 % 42 % 44 % 46 % 48 % 51 % 53 % 55 % 57 % 59 % 61 % 63 % 65 % 68 % 70 % 72 % 74 % 76 % 78 % 80 % 82 % 85 % 87 % 89 % 91 % 93 % 95 % 97 % hh - готово!
superjob - сделано!
hh:  919
superjob:  7


# Домашка урока 3: Сохранение в MongoDB

### Импортируем библиотека и подключаемся к MongoDB

In [72]:
from pymongo import MongoClient
from pprint import pprint

client  = MongoClient('localhost',27017)
db = client['lesson3_hw']
users = db.lesson3_hw

db.drop_collection('lesson3_hw')

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

### п.1: Записываем собранные вакансии в БД MongoDB

In [73]:
data = resultsHH + res_res

j = 0
for i in data:
    try:
        users.insert_one(i)
        j += 1
    except:
        pass
print("Вакансий внесено в Базу: ", j)

# users.insert_many(data)

Вакансий внесено в Базу:  926


### п.2: Выводим вакансии с минимальной зарплатой БОЛЬШЕ введенной пользователем

In [77]:
salary = input("Введите минимальную зарплату: ")
salary = int(salary)

# Сначала ищем все вакансии с зарплатой больше 0 (чтобы сразу исключить вакансии без зарплаты)
objects = users.find({'salary_min':{'$gte':0}})#.sort('author').limit(3)

j = 0
for obj in objects:
    # ищем только вакансии, где валюта 'руб'
    if obj.get('salary_min') >= salary and obj.get('salary_valuta') == 'руб':
        pprint(obj)
        j += 1

print('************************')
print("Вакансий отобранно: ", j)

Введите минимальную зарплату: 0
{'_id': ObjectId('5e3a8c1b1f397ccf6928a0ef'),
 'link': 'https://pushkino.hh.ru/vacancy/35011897?query=Data%20science',
 'name': 'Data Scientist',
 'salary_max': '',
 'salary_min': 200000,
 'salary_valuta': 'руб',
 'site': 'https://hh.ru'}
{'_id': ObjectId('5e3a8c1c1f397ccf6928a0f6'),
 'link': 'https://pushkino.hh.ru/vacancy/35682800?query=Data%20science',
 'name': 'Data Scientist (an International Automotive Startup)',
 'salary_max': 600000,
 'salary_min': 400000,
 'salary_valuta': 'руб',
 'site': 'https://hh.ru'}
{'_id': ObjectId('5e3a8c1c1f397ccf6928a0f7'),
 'link': 'https://pushkino.hh.ru/vacancy/31844648?query=Data%20science',
 'name': 'Junior Quantitative Researcher',
 'salary_max': 250000,
 'salary_min': 250000,
 'salary_valuta': 'руб',
 'site': 'https://hh.ru'}
{'_id': ObjectId('5e3a8c1c1f397ccf6928a0f8'),
 'link': 'https://pushkino.hh.ru/vacancy/35296762?query=Data%20science',
 'name': 'Python Data Scientist',
 'salary_max': '',
 'salary_min': 15

### п.3 А у меня, в данной реализации повторяющиеся записи не добавляются... 